# Hist Design Prototype

This is `fill` method in python loop:

In [1]:
import numpy as np
import numba as nb
from hist import Hist
from hist.axis import Regular

# assets
array = np.random.randn(
    10000,
)
h = Hist.new.Reg(100, -3, 3, name="x", label="x-axis").Double()

In [2]:
# python fill
# h.fill(array)
# h

## Numba: Hist

To extend the Numba, we first need to create a Hist type `HistType` for `Hist`, and then teach Numba about our type inference additions:

In [3]:
from numba import types
from numba.extending import typeof_impl, as_numba_type, type_callable

# create Numba type
class RegularType(types.Type):
    def __init__(self):
        super().__init__(name="Regular")


regular_type = RegularType()

# infer values
@typeof_impl.register(Regular)
def typeof_index(val, c):
    return regular_type


# infer annotations
as_numba_type.register(Regular, regular_type)

# infer operations
@type_callable(Regular)
def type_regular(context):
    def typer(bins, lo, hi):
        if (
            isinstance(bins, types.Integer)
            and isinstance(lo, types.Float)
            and isinstance(hi, types.Float)
        ):
            return regular_type

    return typer

We also need to teach Numba how to actually generate native representation for the new operations:

In [4]:
from numba.core import cgutils
from numba.extending import (
    models,
    register_model,
    make_attribute_wrapper,
    overload_attribute,
    lower_builtin,
    box,
    unbox,
    NativeValue,
)

# define data model
@register_model(RegularType)
class RegularModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ("bins", types.int32),
            ("lo", types.float64),
            ("hi", types.float64),
        ]
        models.StructModel.__init__(self, dmm, fe_type, members)


# expose attributes, porperties and constructors
make_attribute_wrapper(RegularType, "bins", "bins")
make_attribute_wrapper(RegularType, "lo", "lo")
make_attribute_wrapper(RegularType, "hi", "hi")


@overload_attribute(RegularType, "width")
def get_width(reg):
    def getter(reg):
        return (reg.hi - reg.lo) / reg.bins

    return getter


@lower_builtin(Regular, types.Integer, types.Float, types.Float)
def impl_reg(context, builder, sig, args):
    typ = sig.return_type
    lo, hi, bins = args
    reg = cgutils.create_struct_proxy(typ)(context, builder)
    reg.lo = lo
    reg.hi = hi
    reg.bins = bins
    return reg._getvalue()


# unbox and box
@unbox(RegularType)
def unbox_reg(typ, obj, c):
    bins_obj = c.pyapi.object_getattr_string(obj, "bins")
    lo_obj = c.pyapi.object_getattr_string(obj, "lo")
    hi_obj = c.pyapi.object_getattr_string(obj, "hi")
    reg = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    reg.bins = c.pyapi.float_as_double(bins_obj)
    reg.lo = c.pyapi.float_as_double(lo_obj)
    reg.hi = c.pyapi.float_as_double(hi_obj)
    c.pyapi.decref(bins_obj)
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(reg._getvalue(), is_error=is_error)


@box(RegularType)
def box_reg(typ, val, c):
    reg = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    bins_obj = c.pyapi.float_from_double(reg.bins)
    lo_obj = c.pyapi.float_from_double(reg.lo)
    hi_obj = c.pyapi.float_from_double(reg.hi)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(Regular))
    res = c.pyapi.call_function_objargs(class_obj, (bins_obj, lo_obj, hi_obj))
    c.pyapi.decref(bins_obj)
    c.pyapi.decref(lo_obj)
    c.pyapi.decref(hi_obj)
    c.pyapi.decref(class_obj)
    return res

In [5]:
@nb.jit
def nb_create_reg():
    return Regular(
        50,
        -5.0,
        5.0,
    )


nb_create_reg()

<ipython-input-5-4658abf95da1>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function nb_create_reg failed at nopython mode lowering due to: Invalid store of i64 to double in <__main__.RegularModel object at 0x183567040> (trying to write member #1)

File "<ipython-input-5-4658abf95da1>", line 3:
def nb_create_reg():
    return Regular(50, -5., 5.,)
    ^

During: lowering "$10call_function.4 = call $2load_global.0($const4.1, $const6.2, $const8.3, func=$2load_global.0, args=[Var($const4.1, <ipython-input-5-4658abf95da1>:3), Var($const6.2, <ipython-input-5-4658abf95da1>:3), Var($const8.3, <ipython-input-5-4658abf95da1>:3)], kws=(), vararg=None)" at <ipython-input-5-4658abf95da1> (3)
  @nb.jit
/Users/ninolau/anaconda3/envs/hist/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "nb_create_reg" was compiled in object mode without forceobj=True.

File "<ipython-input-5-4658abf95da1>", line 2:
@nb.jit
def n

Regular(50, -5, 5)

In [6]:
# def nb_hist_property(h):
#     print(h.lo)

In [7]:
# Numba fill
# nb_fill(h, array)
# h